In [3]:
import requests

#Nuestras credenciales
client_id = "931d7d4218dc436b9e30b0ad300d487e"
client_secret = "9ebf32e882de4f8aaffd1b3a0d9235c3"

#URL para obtencion de tokens
url = "https://accounts.spotify.com/api/token"
#Headers necesarios para la request
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
#Datos a enviar en la request, aqui van las credenciales
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

#Hacemos la request
response = requests.post(url, headers=headers, data=data)

#Guardamos el token
access_token = response.json().get("access_token")

#Mostramos el resultado
print(access_token)

BQAmWkcPfEj9z2Ynob0fdAf0gMdcWOBRV-wvauTnvVvRu9Naq0ZgnjQeqoNOGpjoFakXm8P_bgZqzAc0-Q7GSsRItshTxCDzKhh13d0HDs9Hc-SRidp1Ekhu0_dhx9BkyuHsf04_nGo


In [4]:
#Biblioteca para formatear json, sólo necesaria por claridad
import json

#Necesitamos la id del artista para poder obtener su información. Para ésto podemos irnos a un artista en spotify, clickear los tres puntos y copiar su id.
artist_id = "1vCWHaC5f2uS3yhpwWbIA6"
url_artist = f"https://api.spotify.com/v1/artists/{artist_id}"
headers_artist = {
    "Authorization": f"Bearer {access_token}"
}

response_artist = requests.get(url_artist, headers=headers_artist)

print(json.dumps(response_artist.json(), indent=4))

{
    "external_urls": {
        "spotify": "https://open.spotify.com/artist/1vCWHaC5f2uS3yhpwWbIA6"
    },
    "followers": {
        "href": null,
        "total": 23530128
    },
    "genres": [
        "edm"
    ],
    "href": "https://api.spotify.com/v1/artists/1vCWHaC5f2uS3yhpwWbIA6",
    "id": "1vCWHaC5f2uS3yhpwWbIA6",
    "images": [
        {
            "url": "https://i.scdn.co/image/ab6761610000e5ebae07171f989fb39736674113",
            "height": 640,
            "width": 640
        },
        {
            "url": "https://i.scdn.co/image/ab67616100005174ae07171f989fb39736674113",
            "height": 320,
            "width": 320
        },
        {
            "url": "https://i.scdn.co/image/ab6761610000f178ae07171f989fb39736674113",
            "height": 160,
            "width": 160
        }
    ],
    "name": "Avicii",
    "popularity": 84,
    "type": "artist",
    "uri": "spotify:artist:1vCWHaC5f2uS3yhpwWbIA6"
}
